<a href="https://colab.research.google.com/github/muruganveera/murugan/blob/master/QlearningwithGYM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import gym
import random
import numpy as np
import time
from gym.envs.registration import register
from IPython.display import clear_output

In [6]:
# https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py
# https://github.com/openai/gym/blob/master/gym/envs/__init__.py
try:
    register(
        id='FrozenLakeNoSlip-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4', 'is_slippery':False},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )
except:
    pass

# env_name = "CartPole-v1"
# env_name = "MountainCar-v0"
# env_name = "MountainCarContinuous-v0"
# env_name = "Acrobot-v1"
# env_name = "Pendulum-v0"
#env_name = "FrozenLake-v0"
env_name = "FrozenLakeNoSlip-v0"
env = gym.make(env_name)
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)
type(env.action_space)

Observation space: Discrete(16)
Action space: Discrete(4)


gym.spaces.discrete.Discrete

In [7]:
class Agent():
    def __init__(self, env):
        self.is_discrete = \
            type(env.action_space) == gym.spaces.discrete.Discrete
        
        if self.is_discrete:
            self.action_size = env.action_space.n
            print("Action size:", self.action_size)
        else:
            self.action_low = env.action_space.low
            self.action_high = env.action_space.high
            self.action_shape = env.action_space.shape
            print("Action range:", self.action_low, self.action_high)
        
    def get_action(self, state):
        if self.is_discrete:
            action = random.choice(range(self.action_size))
        else:
            action = np.random.uniform(self.action_low,
                                       self.action_high,
                                       self.action_shape)
        return action

In [8]:
class QAgent(Agent):
    def __init__(self, env, discount_rate=0.97, learning_rate=0.01):
        super().__init__(env)
        self.state_size = env.observation_space.n
        print("State size:", self.state_size)
        
        self.eps = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.build_model()
        
    def build_model(self):
        self.q_table = 1e-4*np.random.random([self.state_size, self.action_size])
        
    def get_action(self, state):
        q_state = self.q_table[state]
        action_greedy = np.argmax(q_state)
        action_random = super().get_action(state)
        return action_random if random.random() < self.eps else action_greedy
    
    def train(self, experience):
        state, action, next_state, reward, done = experience
        
        q_next = self.q_table[next_state]
        q_next = np.zeros([self.action_size]) if done else q_next
        q_target = reward + self.discount_rate * np.max(q_next)
        
        q_update = q_target - self.q_table[state,action]
        self.q_table[state,action] += self.learning_rate * q_update
        
        if done:
            self.eps = self.eps * 0.99
        
agent = QAgent(env)

Action size: 4
State size: 16


In [9]:
total_reward = 0
for ep in range(100):
    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        agent.train((state,action,next_state,reward,done))
        state = next_state
        total_reward += reward
        
        print("s:", state, "a:", action)
        print("Episode: {}, Total reward: {}, eps: {}".format(ep,total_reward,agent.eps))
        env.render()
        print(agent.q_table)
        time.sleep(0.05)
        clear_output(wait=True)

s: 5 a: 1
Episode: 99, Total reward: 0.0, eps: 0.36603234127322926
  (Down)
SFFF
FHFH
FFFH
HFFG
[[4.07830916e-05 7.67073290e-05 8.57780122e-05 6.66121061e-05]
 [5.20315111e-05 5.62792061e-05 2.64540486e-05 8.38917482e-05]
 [4.84968426e-05 5.71113845e-05 6.79399505e-05 4.00277610e-05]
 [1.65596387e-05 7.09467288e-05 8.46791878e-05 8.46813652e-05]
 [8.81553672e-05 2.68931065e-05 7.00393231e-05 7.53168932e-05]
 [6.27793455e-05 7.60596995e-05 2.97985422e-05 4.62278116e-05]
 [3.91714702e-05 7.70666323e-05 7.01519312e-05 1.28989482e-05]
 [2.83869050e-05 3.53204691e-05 5.59743288e-05 8.33755307e-05]
 [8.67312275e-05 2.44325831e-05 7.28803417e-05 5.64120272e-05]
 [2.53005102e-05 2.97251140e-07 2.45986699e-05 4.21675390e-05]
 [4.83991500e-05 6.69690546e-05 3.33390437e-05 8.76219581e-05]
 [9.60324673e-05 2.99625857e-05 9.32428286e-05 1.36269073e-05]
 [3.50771524e-05 8.37190413e-05 6.04705778e-05 9.85933124e-05]
 [9.32220194e-05 1.61290819e-05 5.39515479e-05 1.03531448e-05]
 [9.08890415e-05 3.543